In [22]:
import os
import pandas as pd
from datetime import datetime

from utils import read_config

from llama_index.core import VectorStoreIndex, StorageContext, Settings
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.vector_stores.types import VectorStoreQuery

import chromadb

from langchain.prompts import PromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)

import google.generativeai as genai
from langchain_google_genai import (
    ChatGoogleGenerativeAI,
    HarmBlockThreshold,
    HarmCategory,
)

In [23]:
info_path = '.env/info.json'
info = read_config(info_path)

# oracle_agent = OracleAgent(info['DW_conn_info'])

API_KEY= info['gkey']
os.environ["GOOGLE_API_KEY"] = API_KEY

In [3]:
model_name = "sentence-transformers/distiluse-base-multilingual-cased-v1"
embed_model = HuggingFaceEmbedding(model_name=model_name)

In [4]:
# load vectordb
db2 = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db2.get_or_create_collection("DashboardQA")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
index = VectorStoreIndex.from_vector_store(
    vector_store,
    embed_model=embed_model,
)


In [145]:
#query = "我要看稅前淨利、損益表、資產負債表的數據要看哪張表"
query = "我要看PM01相關數據"
embedding_vector = embed_model.get_query_embedding(query)
query_vector = VectorStoreQuery(query_embedding=embedding_vector, similarity_top_k=1)

In [146]:
result = vector_store.query(query_vector)

In [147]:
print(result)

VectorStoreQueryResult(nodes=[TextNode(id_='74f8a68c-9a4e-46fe-84c8-d50ebd3c62e7', embedding=None, metadata={'dashboard_name': '投控經營分析', 'report_name': '華紙-營運績效分析日報', 'page_name': '生產面', 'pic_name': '紙機生產指標.jpg'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='797ee95e-af93-4676-8d9a-58ffe3077932', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'dashboard_name': '投控經營分析', 'report_name': '華紙-營運績效分析日報', 'page_name': '生產面', 'pic_name': '紙機生產指標.jpg'}, hash='216c840d57d630a12b8d1d6686b11f06ceaa7bf9f3cda90f4b2c78ad5b8322b4'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='d6d65046-db04-42f2-aee6-805735482914', node_type=<ObjectType.TEXT: '1'>, metadata={'dashboard_name': '投控經營分析', 'report_name': '華紙-營運績效分析日報', 'page_name': '生產面', 'pic_name': '紙機生產指標.jpg'}, hash='28f685075b97c5872a931966d155ede788bcd87a4b65b183f2fb0b55b3106d01')}, text='Answer: \n這張圖顯示了紙漿生產的相關數據，從圖中可以得到以下資訊：\n\n1. **什麼是 PM01？

In [149]:
result.nodes[0].text

'Answer: \n這張圖顯示了紙漿生產的相關數據，從圖中可以得到以下資訊：\n\n1. **什麼是 PM01？** PM01 是什麼機台或生產線的代號？\n2. **每日生產量是多少？** 每日生產量指的是什麼？是重量還是數量？\n3. **生產效率如何？** 生產效率的指標是什麼？\n4. **產出噸數是多少？** 產出噸數指的是什麼產品的產量？\n5. **生產速度是多少？** 生產速度的單位是什麼？\n6. **哪些因素影響生產效率？** 例如停機時間、故障率等等。\n7. **哪些因素影響產出噸數？** 例如原料品質、設備狀態等等。\n8. **如何提高生產效率？** 可以從哪些方面改善生產效率？\n9. **如何提高產出噸數？** 可以從哪些方面提高產出噸數？\n10. **如何降低生產成本？** 可以從哪些方面降低生產成本？'

In [150]:
result.nodes[0].metadata

{'dashboard_name': '投控經營分析',
 'report_name': '華紙-營運績效分析日報',
 'page_name': '生產面',
 'pic_name': '紙機生產指標.jpg'}

### Combine with LangChain

In [151]:
# prompt_template = """
# 根據retrieved中的內容來回答question的問題。如果你不知道答案則說不知道，不要造假答案。

# {retrieved}

# Question: {question}
# Helpful answer:
# """

# PROMPT = PromptTemplate(template=prompt_template, input_variables=['retrieved', 'question'])

# "。如果你不知道答案則說不知道，不要造假答案。"
system_template = """
你現在是專業的儀表板查找跟整理高手，僅需根據Retrieved中的內容來回答question的問題，問題主要是要從哪張儀表表找到滿足Question中的內容，不是在詢問相關數字

例如:
####Question為
# 我要看稅前淨利、損益表、資產負債表的數據要看哪張表時，
####Retrieved為
# dashboard_name: 投控經營分析',
# 'report_name': '華紙-營運績效分析月報',
# 'page_name': '事業部層',
# 'pic_name': '事業部層-完整畫面.jpg''
此時答案是'儀錶板投控經營分析中的華紙-營運績效分析月報，裡面的事業部層分頁'

Retrieved: {retrieved}

Question: {question}
"""

messages = [
    SystemMessagePromptTemplate.from_template(system_template),
    HumanMessagePromptTemplate.from_template("{question}")
]

CHAT_PROMPT = ChatPromptTemplate.from_messages(messages)


In [152]:
llm = ChatGoogleGenerativeAI(
    model="gemini-pro",
    convert_system_message_to_human=True,
    safety_settings={
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
    },
)

In [153]:
chain = CHAT_PROMPT | llm

In [154]:
input_data = {
    "retrieved": result.nodes[0].metadata,
    "question": query
}

In [155]:
input_data

{'retrieved': {'dashboard_name': '投控經營分析',
  'report_name': '華紙-營運績效分析日報',
  'page_name': '生產面',
  'pic_name': '紙機生產指標.jpg'},
 'question': '我要看PM01相關數據'}

In [156]:
llm_response = chain.invoke(input_data)

e:\miniconda3\envs\bibot\lib\site-packages\langchain_google_genai\chat_models.py:350: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


In [157]:
print(llm_response.content)

儀錶板投控經營分析中的華紙-營運績效分析日報，裡面的生產面分頁
